# GenAI model tuning prep data

input format

```
sample_code

commentary

prompt

```


output format

```
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Chelsea Cafe from my current location?"}, {"role": "assistant", "content": "Around five minutes walk. Are you hungry?"}]}
```

```
{"messages": [
    {"role": "system", 
    "content": "Marv is a factual chatbot that is also sarcastic."}, 
    
    {"role": "user", 
    "content": "How far is the Chelsea Cafe from my current location?"}, 
    
    {"role": "assistant", 
    "content": "Around five minutes walk. Are you hungry?"}
    ]}
```

```
{"messages": [
    {"role": "system", 
    "content": "context"}, 
    
    {"role": "user", 
    "content": prompt}, 
    
    {"role": "assistant", 
    "content": sample_code, commentary}
    ]}

```

## for reference here is the openai prompt and result

We tried this with the GOOG chat but it didn't work.

## prompt

```
Write code in python to read a text file with this format:

```
sample_code

commentary

prompt

```
 
and output a jsonl file in this format:

{"messages": [
    {"role": "system", 
    "content": "context"}, 
    
    {"role": "user", 
    "content": prompt}, 
    
    {"role": "assistant", 
    "content": sample_code, commentary}
    ]}
```

## result

```
import json

# Function to read the input text file and convert it to JSONL format
def convert_text_to_jsonl(input_file, output_file):
    messages = []

    with open(input_file, 'r') as file:
        lines = file.readlines()
        if len(lines) != 3:
            print("Input file does not have the correct format (sample_code, commentary, prompt)")
            return

        sample_code = lines[0].strip()
        commentary = lines[1].strip()
        prompt = lines[2].strip()

        messages.append({"role": "system", "content": "context"})
        messages.append({"role": "user", "content": prompt})
        messages.append({"role": "assistant", "content": sample_code, "commentary": commentary})

    with open(output_file, 'w') as jsonl_file:
        for message in messages:
            jsonl_file.write(json.dumps(message) + '\n')

if __name__ == "__main__":
    input_file = "input.txt"  # Replace with your input file's name
    output_file = "output.jsonl"  # Replace with your desired output file's name

    convert_text_to_jsonl(input_file, output_file)

```

In [4]:
import datetime
import pathlib
import json

import pandas as pd
import os
import os.path

In [5]:
# This can be varied to point to different files.
IN_FILE_NAME = "raw.txt"
OUT_FILE_NAME = "processed.jsonl"
os.environ['IN_FILE_NAME'] = IN_FILE_NAME
os.environ['OUT_FILE_NAME'] = OUT_FILE_NAME

In [6]:
# The current directory will be where this src file is located.
# Which is in the notebooks dir of the project
dirpath = os.getcwd()
print("current directory is : " + dirpath)

current directory is : /Users/davis/progs/Keras-Philosophy/notebooks


In [7]:
# Use pathlib to find the root dir of the git repo
root_path = pathlib.PurePath(dirpath).parents[0]
data_path = root_path / 'data'
logs_path = root_path / 'logs'
print("root directory is: ", root_path)
print("data directory is: ",  data_path)
print("logs directory is: ", logs_path)

root directory is:  /Users/davis/progs/Keras-Philosophy
data directory is:  /Users/davis/progs/Keras-Philosophy/data
logs directory is:  /Users/davis/progs/Keras-Philosophy/logs


In [8]:
# Create equivalent dir names in the environment
# Logs
LOGS_DIR_NAME = logs_path.as_posix()
print("LOGS_DIR_NAME: ", LOGS_DIR_NAME)
os.environ['LOGS_DIR_NAME'] = LOGS_DIR_NAME
# Data
DATA_DIR_NAME = data_path.as_posix()
print("DATA_DIR_NAME: ", DATA_DIR_NAME)
os.environ['DATA_DIR_NAME'] = DATA_DIR_NAME

LOGS_DIR_NAME:  /Users/davis/progs/Keras-Philosophy/logs
DATA_DIR_NAME:  /Users/davis/progs/Keras-Philosophy/data


In [9]:
%%bash
# Verify env variables are set
echo ${DATA_DIR_NAME}
echo ${LOGS_DIR_NAME}
echo ${CSV_FILE_NAME}

/Users/davis/progs/Keras-Philosophy/data
/Users/davis/progs/Keras-Philosophy/logs



In [10]:
%%bash
head -n 5 "${DATA_DIR_NAME}/${IN_FILE_NAME}"


SAMPLE_CODE

This is sample code line 1
This is sample code line 2


# routine to build our file via chatGPT

In [69]:
# Function to read the input text file and convert it to JSONL format
def convert_text_to_jsonl(input_file, output_file):
    lines = []
    messages = []
    # stub vars
    sample_code = []
    commentary = []
    prompt = []

    with open(input_file, 'r') as file:
        lines = file.readlines()


    # 
    # find delimiters
    #

    posn = []
    line_nbr = 0
    for a_line in lines:
        #print(a_line)
        if (a_line == "SAMPLE_CODE\n"):
            #print(a_line, " ", line_nbr )
            # save that position
            posn.append(line_nbr) 
        if (a_line == "COMMENTARY\n"):
            #print(a_line)
            posn.append(line_nbr) 
        if (a_line == "PROMPT\n"):
            #print(a_line)
            posn.append(line_nbr) 
        
        line_nbr = line_nbr + 1

    # Record the last line in file
    posn.append(line_nbr) 

    #print("posn: ", posn)

    # separate out the parts
    code_lines = lines[1+1:posn[1]]
    commentary_lines = lines[posn[1]+1:posn[2]]
    prompt_lines = lines[posn[2]+1:posn[3]]

    # dump the parts
    #print("===code_lines:===\n", code_lines)
    #print("===commentary_lines:===\n", commentary_lines)
    #print("===prompt_lines:===\n", prompt_lines)

    a_dict = {}
    a_dict['messages'] = []
    # a_dict['messages'] = [ 
    #     {'role':'system',
    #      'content': 'you are a helpful assistant'},
    #     {'role':'user',
    #      'content': prompt_lines},
    #     {'role':'assistant',
    #      'content': code_lines + ' with commentary: ' + commentary_lines}
    # ]

    code_commentary_lines = code_lines
    code_commentary_lines.append('with commentary: \n')
    # WHY?: using [] or not it appends a generator instead of string.
    #code_commentary_lines.append([str(e) for e in commentary_lines])
    for a_line in commentary_lines:
        code_commentary_lines.append(a_line)
    
    a_dict['messages'].append({'role':'system',
                              'content': 'you are a helpful assistant'})
    a_dict['messages'].append({'role':'user',
                               'content': prompt_lines})
    a_dict['messages'].append({'role':'assistant',
                               'content': code_commentary_lines})


    #print(a_dict)

    with open(output_file, 'w') as jsonl_file:
        jsonl_file.write(json.dumps(a_dict) + '\n')

In [70]:
IN_FQPN = data_path /  pathlib.PurePath(IN_FILE_NAME).as_posix()
OUT_FQPN = data_path /  pathlib.PurePath(OUT_FILE_NAME).as_posix()

convert_text_to_jsonl(IN_FQPN, OUT_FQPN)